In [10]:
import os
######################################################
import csv
import pickle
from copy import deepcopy
from pandas import Series, DataFrame
import pandas as pd

In [11]:
filepath = ''
fastq_path = 'fastq_Tn/'

files = ['ks193','ks195']

skewer_option = '-Q 10'
skewer_adapter = 'AGATCGGAAGAGCACACGTC'

cutadapt_ME_motif = 'GGTTGAGATGTGTATAAGAGACAG'

inpath = fastq_path
outpath = filepath
bowtie_input_extension = '-trimmed-ctadpt.fastq'

mapping = '_5map'
gene_type = '_gene'

In [12]:
skewer = '/home/kazuki/^data_analysis/analtools/skewer022/skewer-0.2.2-linux-x86_64'

log_file = filepath+'log_Tn_skewer.txt'
logs = open(log_file, 'w')
logs.close()

for f in files:
    file_path = fastq_path+f+'.fastq'
    
    command = '%s %s -x %s %s >> %s' % (
        skewer,
        skewer_option,
        skewer_adapter,
        file_path,
        log_file
        ) 
    
    logs = open(log_file, 'a+')
    logs.write('\n\n'+f+'\n')
    logs.close()
    
    os.system(command)

    print f

ks193
ks195


In [13]:
log_file = filepath+'log_Tn_cutadapt.txt'
logs = open(log_file, 'w')
logs.close()

for f in files:
    cutadapt_inputfile = fastq_path+f+'-trimmed.fastq'
    cutadapt_outputfile = fastq_path+f+'-trimmed-ctadpt.fastq'
    
    command = 'cutadapt -g ^%s --discard-untrimmed --minimum-lengt 10 -o %s %s >> %s' % (
        cutadapt_ME_motif,
        cutadapt_outputfile,
        cutadapt_inputfile,
        log_file
        )
    
    logs = open(log_file, 'a+')
    logs.write('\n\n'+f+'\n')
    logs.close()
    
    os.system(command)
    
    print f

ks193
ks195


In [14]:
log_file = filepath+'log_Tn_bowtie.txt'
logs = open(log_file, 'w')
logs.close()

path_chr = outpath + 'alignments/chr/'
path_temp = outpath + 'tmpds/'

if not os.path.exists(outpath + 'alignments/'):
    os.makedirs(outpath + 'alignments/')
if not os.path.exists(path_chr):
    os.makedirs(path_chr)
if not os.path.exists(path_temp):
    os.makedirs(path_temp)

for fname in files:
    bowtie_command = 'bowtie -v 1 -y -m 1 -a --best --strata -S -p 2 '\
        '--un %s%s --max %s%s --al %s%s %s %s%s %s%s 1>> %s 2>> %s'
        
    bowtie_chr = bowtie_command %(
        path_chr,
        fname+'_nomatch.fastq',
        path_chr,
        fname+'_multi.fastq',
        path_chr,
        fname+'_match.fastq',
        'MG1655/e_coli_MG1655',
        inpath,
        fname + bowtie_input_extension,
        path_chr,
        fname + '_match.SAM',
        log_file,
        log_file
        )

    logs = open(log_file, 'a+')
    logs.write('\n\n###################'+fname+'\n')
    logs.write('\n'+bowtie_chr+'\n')
    logs.close()

    os.system(bowtie_chr)
    
    print fname

ks193
ks195


In [15]:
def five_end_mapping(samgen,GFFgen_filename):
    from BCBio import GFF
    from Bio import Seq
    outputdata1={}
    outputdata2={}
    GFFgen=GFF.parse(GFFgen_filename)
    for sequence in GFFgen:
        outputdata1[sequence.id]=[0 for x in range(len(sequence))]
        outputdata2[sequence.id]=[0 for x in range(len(sequence))]
        for read in samgen:
            if read[0][0] == '@':   
                continue
            if read[1] == '4':      
                continue            
            chrom = read[2]             
            readid = read[0]            
            startp = int(read[3]) - 1    
            seq = Seq.Seq(read[9])      
            length = len(seq)           
            if (read[1] == '16'):
                start = startp + length - 1
                outputdata2[chrom][start] += 1
            if (read[1] == '0'):
                start = startp
                outputdata1[chrom][start] += 1
    return [outputdata1,outputdata2] 


def readcounts_to_rpm(readcounts):
    for chrom in readcounts.keys():
        totalread = sum(readcounts[chrom])
        for position in range(len(readcounts[chrom])):
            readcounts[chrom][position]/=float(totalread)
            readcounts[chrom][position]*=1E6            

def writecountsf(rpm,filepath):  
    import struct
    f2=open(filepath+"keys","w")
    for chrom in rpm.keys():
        f=open(filepath+"genome","wb")
        for position in rpm[chrom]:
            f.write(struct.pack("f",position))
        f.close()
        f2.write(chrom+"\n")
    f2.close()

def countstowig(rpm,filestring):
    import random
    f=open(filestring+".wig","w")
    filestring=filestring.partition("_")[0][-3:]

    random.seed(filestring)
    c1=random.randint(0,255)
    random.seed(filestring+"xxx")
    c2=random.randint(0,255)
    random.seed(filestring+"000")
    c3=random.randint(0,255)

    f.write("track name=tracklabel viewLimits=-5:5 color="+str(c1)+','+str(c2)+','+str(c3)+"\n")
    for chrom in rpm.keys():
        if chrom[0:3]=='chr':
            f.write("fixedStep  chrom="+chrom+"  start=1  step=1\n")
        else:
            f.write("fixedStep  chrom=\""+chrom+"\"  start=1  step=1\n")


        for position in rpm[chrom]:
            f.write(str(position)+"\n")
    f.close()

In [16]:
out_path=filepath+'density/'
if not os.path.exists(out_path):
    os.makedirs(out_path)

for f in files:
    samgenmain_filename=filepath+'alignments/chr/'+f+'_match.SAM'
    f_samgenmain=open(samgenmain_filename)
    samgenmain=csv.reader(f_samgenmain,delimiter='\t')

    gff = 'MG1655/coli.gff'

    three_mapped=five_end_mapping(samgenmain,gff) ############################################################
    pickle_readcounts = out_path+f+mapping+'_readcounts.pickle'
    with open(pickle_readcounts, 'wb') as readcounts_f:
        pickle.dump(three_mapped,readcounts_f)

    print f

ks193
ks195


In [17]:
in_path = filepath+'density/'

for f in files:
    readcounts_file = in_path+f+mapping+'_readcounts.pickle'
    with open(readcounts_file, 'rb') as rf:
        readcounts_dict = pickle.load(rf)
    plus_readcounts = readcounts_dict[0][readcounts_dict[0].keys()[0]]
    minus_readcounts = readcounts_dict[1][readcounts_dict[1].keys()[0]]
    totalread = sum(plus_readcounts) + sum(minus_readcounts)

    ######################################################################
    # for readcounts
    both_readcounts_list = []
    for p_rc, n_rc in zip(plus_readcounts,minus_readcounts):
        both_readcounts = p_rc+n_rc
        both_readcounts_list.append(both_readcounts)

    three_mapped_readcounts = []
    for readcounts in readcounts_dict:    
        for chrom in readcounts.keys():
            three_mapped_readcounts.append({chrom:both_readcounts_list})

    pickle_rpm = in_path+f+mapping+'_readcounts_BOTH.pickle'
    with open(pickle_rpm, 'wb') as rpm_f:
        pickle.dump(three_mapped_readcounts,rpm_f)
     
    ######################################################################
    # for rpm
    both_rpm_list = []
    for p_rc, n_rc in zip(plus_readcounts,minus_readcounts):
        both_rpm = p_rc+n_rc
        both_rpm/=float(totalread)
        both_rpm*=1E6  
        both_rpm_list.append(both_rpm)

    three_mapped_rpm = []
    for readcounts in readcounts_dict:    
        for chrom in readcounts.keys():
            three_mapped_rpm.append({chrom:both_rpm_list})

    pickle_rpm = in_path+f+mapping+'_rpm_BOTH.pickle'
    with open(pickle_rpm, 'wb') as rpm_f:
        pickle.dump(three_mapped_rpm,rpm_f)
        
    ######################################################################
    # for wig file
    density_path = in_path+f+mapping
    writecountsf(three_mapped_rpm[0],density_path+"_BOTH_plus_")
    countstowig(three_mapped_rpm[0],density_path+"_BOTH_plus")
    writecountsf(three_mapped_rpm[1],density_path+"_BOTH_minus_")
    countstowig(three_mapped_rpm[1],density_path+"_BOTH_minus")
    
    print f

ks193
ks195


In [18]:
in_path = filepath+'density/'
out_path = filepath+'expression/'
if not os.path.exists(out_path):
    os.makedirs(out_path)

#########################################################################################

with open('MG1655/Ecoli_Gene_TU.pickle','rb') as GT_f:
    Gene_TU_dict = pickle.load(GT_f)

for f in files:
    expression_dict={}
    out_pickle=out_path+f+mapping+gene_type+'_expression_BOTH.pickle'
    out_csv=out_path+f+mapping+gene_type+'_expression_BOTH.csv'
    readcounts_file = in_path+f+mapping+'_readcounts.pickle'
    
    with open(readcounts_file, 'rb') as rf:
        readcounts_dict = pickle.load(rf)
    plus_readcounts = readcounts_dict[0][readcounts_dict[0].keys()[0]]
    minus_readcounts = readcounts_dict[1][readcounts_dict[1].keys()[0]]
    total_read = sum(plus_readcounts)+sum(minus_readcounts)
    
    for key in Gene_TU_dict.keys():
        if key in expression_dict.keys():
            print key +' is overlapping!!!'
            continue
        start = int(Gene_TU_dict[key][2])-1
        end = int(Gene_TU_dict[key][3])
        length = end-start
        if length < 0:
            print key + 'is shorter than zero!!!'
            continue
        if Gene_TU_dict[key][0] == '+':
            reads = float(sum(plus_readcounts[start:end]))
            reads = float(sum(minus_readcounts[start:end]))
        if Gene_TU_dict[key][0] == '-':
            reads = float(sum(plus_readcounts[start:end]))
            reads = float(sum(minus_readcounts[start:end]))
        rpc = (reads/length)*3
        rpkm = (((reads/total_read)*1000000)/length)*1000
        expression_dict[key]=[]
        expression_dict[key].append(reads)
        expression_dict[key].append(rpc)
        expression_dict[key].append(rpkm)
    
    with open(out_pickle, 'wb') as op:
        pickle.dump(expression_dict,op)           
    
    rpkm_DataFrame = DataFrame.from_dict(expression_dict, orient='index')
    rpkm_DataFrame.columns = ['reads','rpc','rpkm']
    rpkm_DataFrame.to_csv(out_csv) 
    
    print f

ks193
ks195
